In [5]:
import numpy as np
import random
import pandas as pd

In [7]:
class GenAlg:

    def __init__(self, population_size, k_crossover, mutation_prob, rates, sigma):
        self.population_size = population_size
        self.k_crossover = k_crossover
        self.mutation_prob = mutation_prob
        self.number_of_stocks = len(rates)

        self.rates = rates.copy()
        self.sigma = sigma.copy()

        self.random_pop()
    


    def random_pop(self):
        self.population = np.random.rand(self.population_size,self.number_of_stocks)
        for  i in range(self.population.shape[0]):
            self.population[i,] = self.normalize(self.population[i,])
    
    def normalize(self,vector):
        sum_of_vector = sum(vector)
        for i in range(len(vector)):
            vector[i] =  vector[i]/sum_of_vector
        return vector
    
    def evaluate(self):
        best_fitness = None
        best = None
        for vector in self.population:
            f = self.fitness(vector)
            if best is None or f > best_fitness:
                best = vector
                best_fitness = f
        return best_fitness,best

    def fitness(self,vector):
        rate = np.sum(vector * self.rates)
        volatility = np.sqrt(np.dot(vector.T,np.dot(self.sigma,vector)))

        return rate/volatility

    def get_lucky_one(self):
        random_num = random.randrange(len(self.population))
        return self.population[random_num,:]
    
    def select_parents(self,lucky_one):
        # if lucky_one exists then remove him
        if lucky_one is not None:
            pop = np.ones(shape=(self.population_size-1,self.number_of_stocks))
            i = 0
            for vec in self.population:
                if not (vec == lucky_one).all():
                    pop[i,:] = vec
                    i+=1
        else:
            pop = self.population.copy()
        
        new_pop = np.array(sorted(pop,key=self.fitness,reverse=True))

        return new_pop[:len(new_pop)//2,:]

    def create_children(self,parents):
        
        children = np.zeros(shape=(self.population_size,self.number_of_stocks))
        np.random.shuffle(parents)
        first_p, second_p = np.split(parents,2)
        for i in range(len(parents)//2):
            parent1 = first_p.pop(0)
            parent2 = second_p.pop(0)

            child1, child2 = self.k_crossover(parent1,parent2)

            child1 = self.mutation(child1)
            child2 = self.mutation(child2)

            children[i,:] = child1
            children[i+len(parents)//2,:] = child2

        return children 
    
    def k_crossover(self,parent1,parent2):
        pass

    def solve(self,max_generations, goal):
        
        best_fit, best = self.evaluate()
        
        for _ in max_generations:
            if best_fit > goal:
                break

            lucky_one = None
            if len(self.population)%2 == 1:
                lucky_one = self.get_lucky_one()

            parents = self.select_parents(lucky_one)

            children = self.create_children(parents)

            if lucky_one is not None:
                lucky_one = lucky_one.reshape((1,len(lucky_one)))
            self.population = np.concatenate((lucky_one,parents,children),axis=0)

            best_fit, best = self.evaluate()
        return best,best_fit

In [8]:
wanted_stocks = ['GOOG', 'SPG', 'GOOGL', 'MSFT', 'GD', 'ACN', 'COP', 'F', 'BAC', 'GS',
                'NVDA', 'AIG', 'MS', 'WFC', 'ORCL', 'XOM', 'TGT', 'LOW', 'EXC', 'COST',
                'AXP', 'BK', 'JPM', 'COF', 'CSCO', 'DHR', 'UNH', 'CVS', 'LLY', 'CVX',
                'MET', 'AMT', 'CRM', 'BLK', 'RTX', 'MCD', 'TMO', 'LIN', 'ADBE', 'EMR',
                'USB', 'UPS', 'TSLA', 'PFE', 'PM']
stocks = pd.read_csv('data/sap100_data_08112021.csv',index_col=0).loc[:,wanted_stocks]
returns = stocks/stocks.shift(1)-1
rates = returns.mean() * 252
sigma = returns.cov() * 252
g = GenAlg(5, 10, 0, rates,sigma)


test_pop = np.zeros(shape=(5,45))
for i in range(5):
    test_pop[i,i] = 1
print(test_pop)
for i in range(5):
    print(g.fitness(test_pop[i,:]))
g.population = test_pop
v = g.get_lucky_one()
print('lucky one: ',v)
print('paretns')
print(g.select_parents(v))

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
2.6472751691283403
2.6205233851220497
2.5800016932008325
2.551735593150768
2.534202575387018
lucky one:  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
paretns
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0